# Notebook sur l'inférence en 8bit, fine tuning avec LoRA

## Fine-tune large models using 🤗 [`peft`](https://github.com/huggingface/peft) adapters, [`transformers`](https://github.com/huggingface/transformers) & [`bitsandbytes`](https://github.com/TimDettmers/bitsandbytes)

Dans ce tutoriel, nous verrons une technique de fine-tuning de LLMs avec la librairie `peft`. Avec `bitsandbytes` on peut charger notre LLM en **8-bit**.

The fine-tuning method will rely on a recent method called "Low Rank Adapters" ([LoRA](https://arxiv.org/pdf/2106.09685.pdf)), instead of fine-tuning the entire model you just have to fine-tune these adapters and load them properly inside the model.

Code tiré de HuggingFace : https://huggingface.co/bigscience/bloomz-3b
Source pour la génération (inférence) https://huggingface.co/docs/transformers/main_classes/text_generation

### Install requirements

First, run the cells below to install the requirements:

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib einops
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [ ]:
!nvidia-smi

Tue Jun 27 13:42:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
# spécifier l'identifiant du périphérique CUDA à utiliser (calcul sur GPU)
# seul le premier périphérique CUDA disponible sera utilisé pour les calculs
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, AutoModelForSeq2SeqLM

In [ ]:
free_in_GB = int(torch.cuda.mem_get_info()[0] / 1024**3) # Convertion bytes en GigaBytes (/1024^3)
max_memory = f"{free_in_GB-2}GB"

n_gpus = torch.cuda.device_count() # 1 GPU à notre disposition
max_memory = {i: max_memory for i in range(n_gpus)}
max_memory

{0: '12GB'}

### Model loading : bloomz-3b

Import du model et tokenizer. A faire tourner sur GPU.

In [ ]:
name = 'bigscience/bloomz-1b1'  # bigscience/bloomz-3b, tiiuae/falcon-7b # bloomz-560m bloomz-1b1 bloomz-1b7 bloomz-3b bloomz-7b1

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=name,  # nom du modèle sur HuggingFace ou chemin
    load_in_8bit=True,      # loader en 8 ou 4 bit avec bitsandbytes
    device_map={"":0},      # ou "auto", pour répartition sur GPU
    trust_remote_code=True, #  Whether or not to allow for custom models defined on the Hub in their own modeling files. This option should only be set to True for repositories you trust and in which you have read the code, as it will execute code present on the Hub on your local machine.
)

tokenizer = AutoTokenizer.from_pretrained(name)

### Inférence

Une fois le modèle chargé, on peut l'inférer. De manière rapide (sans paramètres)

In [ ]:
inputs = tokenizer.encode("What is capitale of France ?", return_tensors="pt").to('cuda:0') # spécifier sur GPU : le numéro 0
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


What is capitale of France ? Paris</s>


Ou bien avec plus de paramètres :

La fonction `torch.cuda.amp.autocast()` est utilisée dans PyTorch pour effectuer un calcul de précision mixte automatique sur les opérations GPU. L'utilisation de la précision mixte est une technique courante pour accélérer les calculs tout en maintenant une précision acceptable.

Lorsque vous utilisez `torch.cuda.amp.autocast()`, les opérations à virgule flottante effectuées à l'intérieur du bloc indenté sont automatiquement converties en précision réduite (FP16) lorsque cela est possible. Cela permet d'exploiter les avantages de performance offerts par les calculs en demi-précision.

In [ ]:
batch = tokenizer(
    """Explain AI.
    """,
    padding=True,
    truncation=True,
    return_tensors='pt'
)
batch = batch.to('cuda:0')
with torch.cuda.amp.autocast():
    output_tokens = model.generate(
        input_ids = batch.input_ids,
        max_new_tokens=75,
        temperature=0.7, # The value used to modulate the next token probabilities. A Temperature of 0 makes the model deterministic.
        top_p=0.7,       # If set to float < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id, # The id of the padding token.
        eos_token_id=tokenizer.eos_token_id, # The id of the end-of-sequence token. Optionally, use a list to set multiple end-of-sequence tokens.
    )
generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

# Inspect message response in the outputs
print(generated_text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Explain AI.
     The AI is a computer program that is used to solve problems. It is a computer program that is used to solve problems.


### Prepare model for training

Some pre-processing needs to be done before training such an int8 model using `peft`, therefore let's import an utiliy function `prepare_model_for_kbit_training` that will:
- Casts all the non `int8` modules to full precision (`fp32`) for stability
- Add a `forward_hook` to the input embedding layer to enable gradient computation of the input hidden states
- Enable gradient checkpointing for more memory-efficient training

In [ ]:
# préparer un modèle pour l'entraînement à une précision réduite (k-bit training)
from peft import prepare_model_for_kbit_training

# gradient checkpointing : économise de la mémoire lors de la rétropropagation du gradient dans les DNN.
# (recalcule certaines activations intermédiaires plutôt que stocker) --> diminue les exigences de mémoire, mais augmente le temps de calcul.
model.gradient_checkpointing_enable()
# préparer le modèle à l'entraînement à une précision réduite (k-bit training) réduire la précision des poids
model = prepare_model_for_kbit_training(model)

### Apply LoRA

Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

In [ ]:
# fonction qui permet d'observer combien de paramètres seront mis à jour
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

Paramètres de `peft.LoraConfig()`pour la configuration de LoRA.

source : https://huggingface.co/docs/peft/conceptual_guides/lora

*    r: the rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters.
*   alpha: LoRA scaling factor.
*  target_modules: The modules (for example, attention blocks) to apply the LoRA update matrices.
*  bias: Specifies if the bias parameters should be trained. Can be 'none', 'all' or 'lora_only'.
*  modules_to_save: List of modules apart from LoRA layers to be set as trainable and saved in the final checkpoint. These typically include model’s custom head that is randomly initialized for the fine-tuning task.
*    layers_to_transform: List of layers to be transformed by LoRA. If not specified, all layers in target_modules are transformed.
*    layers_pattern: Pattern to match layer names in target_modules, if layers_to_transform is specified. By default PeftModel will look at common layer pattern (layers, h, blocks, etc.), use it for exotic and custom models.`

In [ ]:
from peft import LoraConfig, get_peft_model
# crée une instance de la classe LoraConfig avec para de config pour le modèle PEFT (objet)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"], # les matrices de poids Q, K, V sont modifiés
    lora_dropout=0.05,
    bias="none",
    # CAUSAL_LM : causal language modeling, SEQ_CLS : sequence classification, TOKEN_CLS : token classification, SEQ_2_SEQ_LM : sequence-to-sequence language modeling
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 2359296 || all params: 1067673600 || trainable%: 0.22097539922313336


In [ ]:
# Verifying the datatypes.
dtypes = {}
for _, p in model.named_parameters():
    dtype = p.dtype
    if dtype not in dtypes:
        dtypes[dtype] = 0
    dtypes[dtype] += p.numel()
total = 0
for k, v in dtypes.items():
    total += v
for k, v in dtypes.items():
    print(k, v, v / total)

torch.float32 388196352 0.3635908502373759
torch.int8 679477248 0.6364091497626241


### Load dataset

On souhaite fine-tuner notre modèle sur une tâche spécifique : classification de sentiments de commentaires de films en français.

In [ ]:
import transformers
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np

In [ ]:
# extract, load, and transform OpenAssistant/oasst1 chatbot dataset

# set some pandas options to make the output more readable
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

Exemple sur le Dataset Allociné : classification de commentaires : positif/négatif.

In [ ]:
# load dataset from huggingface datasets
ds = load_dataset('allocine') # ['allocine', "OpenAssistant/oasst1", 'etalab-ia/piaf']

In [ ]:
# lets convert the train dataset to a pandas df
df = ds['train'].to_pandas()[:1000] #ds['train'].to_pandas()
df['review'] = df['review'].apply(lambda x : '<human>: '+str(x))
df['label'] = df['label'].apply(lambda x : '<bot>: Negative' if x == 0 else '<bot>: Positive')
df['new'] = df['review'] + df['label']
df.drop(['review', 'label'], inplace=True, axis=1)
df.head()

,new
0,<human>: Si vous cherchez du cinéma abrutissan...
1,"<human>: Trash, re-trash et re-re-trash...! Un..."
2,"<human>: Et si, dans les 5 premières minutes d..."
3,<human>: Mon dieu ! Quelle métaphore filée ! J...
4,"<human>: Premier film de la saga Kozure Okami,..."


On met en forme la donnée pour quelle ressemble à :   
human: commentaire_de_film. bot: Négatif_ou_positif

Pourquoi ? Il s'agit d'apprentissage supervisé, donc en introduisant ces balises là, on permet au modèle d'intégrer la manière dont on souhaite qu'il réagisse la prochaine fois.


NB : Il s'agit d'un exemple illustratif pour montrer comment le faire en pratique. Il peut exister d'autres méthodes que le fine-tuning pour cette tâche.

In [ ]:
df.loc[0, 'new']

'<human>: Si vous cherchez du cinéma abrutissant à tous les étages,n\'ayant aucune peur du cliché en castagnettes et moralement douteux,"From Paris with love" est fait pour vous.Toutes les productions Besson,via sa filière EuropaCorp ont de quoi faire naître la moquerie.Paris y est encore une fois montrée comme une capitale exotique,mais attention si l\'on se dirige vers la banlieue,on y trouve tout plein d\'intégristes musulmans prêts à faire sauter le caisson d\'une ambassadrice américaine.Nauséeux.Alors on se dit qu\'on va au moins pouvoir apprécier la déconnade d\'un classique buddy-movie avec le jeune agent aux dents longues obligé de faire équipe avec un vieux lou complètement timbré.Mais d\'un côté,on a un Jonathan Rhys-meyers fayot au possible,et de l\'autre un John Travolta en total délire narcissico-badass,crâne rasé et bouc proéminent à l\'appui.Sinon,il n\'y a aucun scénario.Seulement,des poursuites débiles sur l\'autoroute,Travolta qui étale 10 mecs à l\'arme blanche en 8 

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data = Dataset.from_pandas(df)

data = data.map(lambda samples: tokenizer(samples["new"], padding=True, truncation=True,), batched=True)
data

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['new', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

Exemple sur question réponses FR. PIAF.

In [ ]:
# load dataset from huggingface datasets
ds1 = load_dataset('etalab-ia/piaf') # ['allocine', "OpenAssistant/oasst1", 'etalab-ia/piaf']


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3835 [00:00<?, ? examples/s]

Dataset piaf downloaded and prepared to /root/.cache/huggingface/datasets/etalab-ia___piaf/plain_text/1.0.0/535c60f4155fe1d644c1746e86131963c082f309d0dbb5ba4d606786c7f4a6ae. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Autre exemple : fine-tuning sur de l'extraction d'information.

NB : Il s'agit d'un exemple illustratif pour montrer comment le faire en pratique. Il peut exister d'autres méthodes que le fine-tuning pour cette tâche.

In [ ]:
# lets convert the train dataset to a pandas df
df1 = ds1["train"].to_pandas()
df1.drop(['id', 'title'], axis=1, inplace=True)
data_raw = []
for i in range(len(df)):
  data_raw.append(f"<context>:{df1.loc[i,'context']}\n<question>:{df1.loc[i,'question']}\n<answers>:{df1.loc[i,'answers']['text'][0]}")
datadf = pd.DataFrame(data_raw, columns=['text'])
data1 = Dataset.from_pandas(datadf)
data1

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data1 = data1.map(lambda samples: tokenizer(samples["text"], padding=True, truncation=True,), batched=True)
data1

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [ ]:
data1['text'][0]

"<context>:Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 milliards d'euros en 2003 et 12 milliards d'euros en 2019), contre 7,9 milliards d'euros pour les collectivités locales, 3,2 pour l'État, et 2,2 pour les entreprises. Parmi les dépenses sportives des ménages en 2003, 3,7 milliards sont consacrés aux vêtements de sport et chaussures, 2 aux biens durables, 2,7 aux autres biens et 5,8 aux services. Le Ministère de la Jeunesse et des Sports estime à 100 000 (58 % d'hommes pour 42 % de femmes) le nombre de salariés travaillant pour le secteur sportif en France pour quelque 20 000 employeurs.\n<question>:Combien de personnes travaillent au ministère des sports\n<answers>:100 000"

In [ ]:
print(datadf.loc[0, 'text'])

<context>:Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 milliards d'euros en 2003 et 12 milliards d'euros en 2019), contre 7,9 milliards d'euros pour les collectivités locales, 3,2 pour l'État, et 2,2 pour les entreprises. Parmi les dépenses sportives des ménages en 2003, 3,7 milliards sont consacrés aux vêtements de sport et chaussures, 2 aux biens durables, 2,7 aux autres biens et 5,8 aux services. Le Ministère de la Jeunesse et des Sports estime à 100 000 (58 % d'hommes pour 42 % de femmes) le nombre de salariés travaillant pour le secteur sportif en France pour quelque 20 000 employeurs.
<question>:Combien de personnes travaillent au ministère des sports
<answers>:100 000


# Training
Source : https://huggingface.co/dfurman/falcon-7b-chat-oasst1/blob/main/finetune_falcon7b_oasst1_with_bnb_peft.ipynb

Paramètres de `transformers.TrainingArguments()` pour l'entraînement.

source : https://huggingface.co/docs/transformers/main_classes/trainer

* auto_find_batch_size : Whether to find a batch size that will fit into memory automatically through exponential decay, avoiding CUDA Out-of-Memory errors. Requires accelerate to be installed (pip install accelerate)
* gradient_accumulation_steps : Number of updates steps to accumulate the gradients for, before performing a backward/update pass.
* num_train_epochs : Total number of training epochs to perform.
* learning_rate : The initial learning rate for AdamW optimizer
* fp16 : Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training.
* save_total_limit : If a value is passed, will limit the total amount of checkpoints. Deletes the older checkpoints in output_dir.
* logging_steps : Number of update steps between two logs if logging_strategy="steps". Should be an integer or a float in range [0,1). If smaller than 1, will be interpreted as ratio of total training steps.
* output_dir : The output directory where the model predictions and checkpoints will be written.
* save_strategy :  The checkpoint save strategy to adopt during training. Possible values are: "no": No save is done during training."epoch": Save is done at the end of each epoch."steps": Save is done every save_steps.
* optim : The optimizer to use: adamw_hf, adamw_torch, adamw_torch_fused, adamw_apex_fused, adamw_anyprecision or adafactor.
* lr_scheduler_type : The scheduler type to use. See the documentation of SchedulerType for all possible values.
* warmup_ratio : Ratio of total training steps used for a linear warmup from 0 to learning_rate.


In [ ]:
training_args = transformers.TrainingArguments(
    auto_find_batch_size=True,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=4,
    logging_steps=25,
    output_dir="./outputs",
    save_strategy='epoch',
    optim="paged_adamw_8bit",
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
25,3.478800
50,3.204800


TrainOutput(global_step=62, training_loss=3.3170780058830016, metrics={'train_runtime': 546.3201, 'train_samples_per_second': 1.83, 'train_steps_per_second': 0.113, 'total_flos': 1876265191342080.0, 'train_loss': 3.3170780058830016, 'epoch': 0.99})

# Save model (trainer)

Rappel de peft :


```
peft_config = get_peft_config(config)
model = AutoModelForCausalLM.from_pretrained("gpt2-large")
peft_model = PeftModelForCausalLM(model, peft_config)
peft_model.print_trainable_parameters()
```



In [ ]:
# Création des fichiers adapter_config.json, adapter_model.bin
model.save_pretrained("path/to/model")

In [ ]:
# load the model
lora_config = LoraConfig.from_pretrained("path/to/model")
m = get_peft_model(model, lora_config)

# Inference


**Comparaison des résultats**
Un exemple avec :

* \<context>:Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 milliards d'euros en 2003 et 12 milliards d'euros en 2019), contre 7,9 milliards d'euros pour les collectivités locales, 3,2 pour l'État, et 2,2 pour les entreprises. Parmi les dépenses sportives des ménages en 2003, 3,7 milliards sont consacrés aux vêtements de sport et chaussures, 2 aux biens durables, 2,7 aux autres biens et 5,8 aux services. Le Ministère de la Jeunesse et des Sports estime à 100 000 (58 % d'hommes pour 42 % de femmes) le nombre de salariés travaillant pour le secteur sportif en France pour quelque 20 000 employeurs.


* \<question>:Combien de personnes travaillent au ministère des sports

Sans fine-tuning :
* \<answers>:En France, le ministère des sports est composé de trois ministères : le ministère de la Jeunesse et des Sports, le ministère de la Culture et de la Communication et le ministère de l'Éducation nationale. Le ministère de la Culture et de la Communication est chargé de la culture, de la communication et de l'éducation. Le ministère de l'Éducation nationale est chargé de l'enseignement supérieur et de la recherche. Le ministère de la Jeunesse et des Sports est chargé de la jeunesse, de la famille, de la culture

Avec fine-tuning :
* \<answers après LoRA>:20 000 employeurs